In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
mnist=input_data.read_data_sets("MNIST_data",one_hot=True)

#每个批次的大小
batch_size=100
#计算一共有多少个批次
a_batch=mnist.train.num_examples//batch_size

#初始化权重
def weight_variable(shape,name):
    initial=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial,name=name)

#初始化偏置
def bias_variable(shape,name):
    initial=tf.constant(0,1,shape=shape)
    return tf.Variable(initial,name=name)

#卷积层
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#池化层
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],
                          strides=[1,2,2,1],padding="SAME")

#命名空间
with tf.name_scope("input"):
    #定义两个placeholder
    x=tf.placeholder(tf.float32,[None,784],name="x_input")
    y=tf.placeholder(tf.float32,[None,10],name="y_input")
    with tf.name_scope("x_image"):
        x_image=tf.reshape(x,[-1,28,28,1],name="x_image")

with tf.name_scope("Conv1"):
    #初始化第一个卷积层的权重和偏置
    with tf.name_scope("W_conv1"):
        W_conv1=weight_variable([5,5,1,10],name="W_conv1")
        #5*5的采样窗口，6个卷积核从一个平面抽取特征
    with tf.name_scope("b_conv1"):
        b_conv1=bias_variable([10],name="b_conv1")
    with tf.name_scope("relu"):
        h_conv1=tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
    with tf.name_scope("h_pool1"):
        h_pool1=max_pool_2x2(h_conv1)
        #池化

with tf.name_scope("Conv2"):
    with tf.name_scope("W_conv2"):
        W_conv2=weight_variable([5,5,10,10],name="W_conv2")
    with tf.name_scope("b_conv2"):
        b_conv2=bias_variable([10],name="b_conv2")
    with tf.name_scope("relu"):
        h_conv2=tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
    with tf.name_scope("h_pool2"):
        h_pool2=max_pool_2x2(h_conv2)

#28*28的图片第一次卷积后是28*28，第一次池化后是14*14
#第二次卷积后是14*14，第二次池化后是7*7

with tf.name_scope("fe1"):
    #定义第一个全连接层的权重
    with tf.name_scope("W_fe1"):
        W_fe1=weight_variable([7*7*10,60],name="W_fe1")
    with tf.name_scope("b_fe1"):
        b_fe1=bias_variable([60],name="b_fe1")
    
    with tf.name_scope("h_pool2_flat"):
        #把池化层2的输出扁平化
        h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*10],name="h_pool2_flat")
    with tf.name_scope("relu"):
        h_fel=tf.nn.relu(tf.matmul(h_pool2_flat,W_fe1)+b_fe1)
   
    with tf.name_scope("keep_prob"):
        keep_prob=tf.placeholder(tf.float32,name="keep_prob")
    with tf.name_scope("h_fe1_drop"):
        h_fe1_drop=tf.nn.dropout(h_fel,keep_prob,name="h_fe1_drop")

with tf.name_scope("fe2"):
    #定义第二个全连接层
    with tf.name_scope("W_fe2"):
        W_fe2=weight_variable([60,10],name="W_fe2")
    with tf.name_scope("b_fe2"):
        b_fe2=bias_variable([10],name="b_fe2")

    with tf.name_scope("softmax"):
        #计算输出
        prediction=tf.nn.softmax(tf.matmul(h_fe1_drop,W_fe2)+b_fe2)

with tf.name_scope("cross_entropy"):
    #交叉熵代价函数
    cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits
                             (labels=y,logits=prediction),name="cross_entropy")
    tf.summary.scalar("cross_entropy",cross_entropy)

with tf.name_scope("train"):
    #优化
    train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

with tf.name_scope("accuracy"):
    with tf.name_scope("correct_prediction"):
        #结果存放在一个布尔型列表中
        correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
    with tf.name_scope("accuracy"):
        #求准确率
        accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar("accuracy",accuracy)

#合并所有的summary
merged=tf.summary.merge_all()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer=tf.summary.FileWriter("logs/train",sess.graph)
    test_writer=tf.summary.FileWriter("logs/test",sess.graph)
    
    for i in range(500):
        #训练模型
        batch_xs,batch_ys=mnist.train.next_batch(batch_size)
        sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,
                                           keep_prob:0.5})
        #记录训练集计算的参数
        summary=sess.run(merged,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1})
        train_writer.add_summary(summary,i)
        
        #记录测试集计算的参数
        batch_xs,batch_ys=mnist.test.next_batch(batch_size)
        summary=sess.run(merged,feed_dict={x:batch_xs,y:batch_ys,keep_prob:1})
        test_writer.add_summary(summary,i)
        
        if i%10==0:
            test_acc=sess.run(accuracy,feed_dict={x:mnist.test.images,
                                         y:mnist.test.labels
                                         ,keep_prob:1.0})
            train_acc=sess.run(accuracy,feed_dict={x:mnist.train.images[:10000],
                                         y:mnist.train.labels[:10000]
                                         ,keep_prob:1.0}) 
            print('iter',str(i),'testing Accuracy',str(test_acc),
                  "Training Accuaracy",str(train_acc))
          

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

iter 0 testing Accuracy 0.0858 Training Accuaracy 0.086
iter 10 testing Accuracy 0.1192 Training Accuaracy 0.120